How to install the package

- Open VS Code terminal in the project root:
- Run:

```
cd "c:\EC\BSE\DSDM\Term 1\21DM004 Computing for Data Science\hw5\process_data_project"
python -m pip install -e .
```

In [5]:
%pip install -U git+https://github.com/ElvisCasco/hw5.git@main

  Cloning https://github.com/ElvisCasco/hw5.git (to revision main) to c:\users\ecasc\appdata\local\temp\pip-req-build-1xcmrmkw
  Resolved https://github.com/ElvisCasco/hw5.git to commit daf4355761da12511f39921cf22609d034828db6
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for process_data: filename=process_data-0.1.0-py3-none-any.whl size=7722 sha256=ec5e695d930471a52eec83086adce99eeecb644529ac25d014442343b7fd615d
  Stored in directory: C:\Users\ecasc\AppData\Local\Temp\pip-ephem-wheel-cache-3irft8cm\wheels\ba\14\d0\f77d3df0a4f426f41340c9fb109952daa324eafc65e7cce0a1
Successfully built process_data
  Attempting uninstall: process_data
    Found existing installation: process-data 0.3
    Unin

  Running command git clone --filter=blob:none --quiet https://github.com/ElvisCasco/hw5.git 'C:\Users\ecasc\AppData\Local\Temp\pip-req-build-1xcmrmkw'
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
hw5 0.1.0 requires process-data>=0.3, but you have process-data 0.1.0 which is incompatible.


In [9]:
from process_data import (
    CSVDataLoader, DataPreprocessor,
    BMICalculator, EthnicityEncoder, GenderBinaryEncoder,
    DiabetesModel,
)
from sklearn.metrics import roc_auc_score

In [10]:
file_name = "C:/EC/BSE/DSDM/Term 1/21DM004 Computing for Data Science/hw5/process_data_project/process_data/data/sample_diabetes_mellitus_data.csv"

loader = CSVDataLoader(file_name)
train_df, test_df = loader.split_data()

pre = DataPreprocessor(train_df, test_df)
train_clean, test_clean = (
    pre.remove_nans(['age','gender','ethnicity'])
      .fill_nans_with_mean(['height','weight'])
      .get_data()
)

for t in [BMICalculator(), EthnicityEncoder(), GenderBinaryEncoder()]:
    train_clean = t.transform(train_clean)
    test_clean = t.transform(test_clean)


Data loaded. Shape: (10000, 53)

Data split: 
Train ((8000, 53)), 
Test ((2000, 53))

Data after removing rows with NaNs in ['age', 'gender', 'ethnicity']: 
Train ((7494, 53)), 
Test ((1874, 53))

Filled NaNs in ['height', 'weight'] with mean.



In [11]:
feature_cols = ['age', 'bmi', 'gender_M', 'gender_F'] + [c for c in train_clean.columns if c.startswith('ethnicity_')]
target_col = 'diabetes_mellitus'

model = DiabetesModel(feature_columns=feature_cols, target_column=target_col, hyperparameters={'n_estimators':200,'max_depth':10,'random_state':42})
model.train(train_clean)

probs = model.predict(test_clean)
pos = probs[:,1] if probs.ndim == 2 else probs
test_clean = test_clean.copy()
test_clean['predictions_prob'] = pos
test_clean['predictions'] = (pos >= 0.5).astype(int)

roc_auc_score(test_clean[target_col], test_clean['predictions_prob'])

0.661790135180888